<span style="color: green;">Day 10 - Advent of Code 2023:</span>
<br>--- Part 1: ---
<br>Constant & function definitions

In [574]:
from itertools import combinations
import numpy as np

# Definitions section
# "PIPES" is defined, in the order:
# 'Up Down', 'Up Left', 'Up Right', 'Down Left', 'Down Right', 'Left Right'
PIPES             = ['|', 'J', 'L', '7', 'F', '-']
COMBINATIONS      = list(combinations(['Up', 'Down', 'Left', 'Right'], 2))
COMBINATIONS_DICT = {key: value for value, key in zip(COMBINATIONS, PIPES)}; COMBINATIONS_DICT['S'] = ('Up', 'Right', 'Down', 'Left'); COMBINATIONS_DICT['.'] = ''
d                 = {'Up': 'Down', 'Down': 'Up', 'Left': 'Right', 'Right': 'Left'}
ROTATIONS         = {
                        ('Right', 'Up'): 'Out',
                        ('Right', 'Down'): 'In',
                        ('Left', 'Up'): 'In',
                        ('Left', 'Down'): 'Out',
                        ('Up', 'Left'): 'Out',
                        ('Up', 'Right'): 'In',
                        ('Down', 'Right'): 'Out',
                        ('Down', 'Left'): 'In'
                    }

# Function definitions
def locate_element(matrix, element):
    # Used for getting the position of the initial 'S' element
    for i, row in enumerate(matrix):
        for j, value in enumerate(row):
            if value == element:
                return (i, j)
    return None

def get_neighbours(matrix, pos, diag = False):
    i, j, neighbours = pos[0], pos[1], []
    # Clean this up later, it's kind of ugly.
    if i > 0:                  neighbours.append((i-1, j))
    if j < len(matrix[i]) - 1: neighbours.append((i, j+1))
    if i < len(matrix) - 1:    neighbours.append((i+1, j))
    if j > 0:                  neighbours.append((i, j-1))

    if(diag == True):
        # Check if the diagonal positions are within the grid boundaries
        if i < len(matrix) - 1 and j < len(matrix[i]) - 1: neighbours.append((i+1, j+1))
        if i < len(matrix) - 1 and j > 0:                  neighbours.append((i+1, j-1))
        if i > 0 and j < len(matrix[i]) - 1:               neighbours.append((i-1, j+1))
        if i > 0 and j > 0:                                neighbours.append((i-1, j-1))
    return neighbours

def get_direction(pos_i, pos_f): # get direction in which we just moved.
    if pos_i[0] == pos_f[0]: return 'Right' if pos_i[1] < pos_f[1] else 'Left'
    else:                    return 'Down'  if pos_i[0] < pos_f[0] else 'Up'

[Part 1] Actual algorithm:

In [575]:
with open('Input.txt', 'r') as file:
    L     = file.read().split()
    
mountain_grid, direction_grid = np.zeros((len(L), len(L[0]))), np.zeros((len(L), len(L[0]))).tolist()
pos = locate_element(L, 'S'); ctr, previous_direction, F, path_positions= 1, None, False, [pos]
while(F == False):
    neighbours = get_neighbours(L, pos)
    for pos_N in neighbours:
        direction = get_direction(pos, pos_N); direction_grid[pos[0]][pos[1]] = direction
        if(     (d[direction] in COMBINATIONS_DICT[L[pos_N[0]][pos_N[1]]]) and \
                (  direction  in COMBINATIONS_DICT[L[pos[0]][pos[1]]])     and \
                (d[direction] != previous_direction) ):
            ctr += 1; pos, previous_direction, mountain_grid[pos] = pos_N, direction, ctr; path_positions.append(pos)
            if(L[pos[0]][pos[1]] == 'S'): mountain_grid[pos], F = 1, True; break
            break
        
print(f'Number of steps to the farthest position = {ctr // 2}.')

Number of steps to the farthest position = 6886.


--- Part 2: ---

Constant & function definitions

In [576]:
# Function definitions, for part 2
def repopulate(m, n, v, r_flag, diag = True):
    L, T = [(m, n)], [[(m, n)]]
    while(True):
        Nn = []
        for k in L:
            N, mountain_grid[k] = [pos_N for pos_N in get_neighbours(mountain_grid, k, diag) if mountain_grid[pos_N] == 0], v
            for pos_N in N: mountain_grid[pos_N] = v
            Nn.append(N)
        L = [item for sublist in Nn for item in sublist]; T.append(L)
        if(len(L) == 0): break
    return [item for sublist in T for item in sublist] if r_flag else None

[Part 2] Actual algorithm:

In [577]:
# Repopulating to '-1', from each corner. Aka, getting rid of the 'obvious' "Outsides"
L_m, L_n = len(mountain_grid) - 1, len(mountain_grid[0]) - 1
repopulate(0, 0, -1, False); repopulate(L_m, L_n, -1, False); repopulate(0, L_n, -1, False); repopulate(L_m, 0, -1, False)
in_Vec = []
S = 0
for m in range(len(mountain_grid)):
    for n in range(len(mountain_grid[0])):
        if(mountain_grid[m, n] == 0):
            R = repopulate(m, n, -10, True); seq = []
            for r in R:
                seq.append([pos_N for pos_N in get_neighbours(mountain_grid, r, diag = True) if mountain_grid[pos_N] > 0])
            seq = sorted(set([item for sublist in seq for item in sublist]), key=lambda x: path_positions.index(x)); cycle = []
            print(f'SEQ = {seq}.')
            for Pp in seq:
                cycle.append(direction_grid[Pp[0]][Pp[1]])
            start = cycle[0]; print(f'SEQ = {seq}.')
            for i, Dd in enumerate(cycle): # Finds if the "circle path" is circulated in the 'clockwise' or 'counter-clockwise' direction.
                print(f'position = {path_positions[path_positions.index(seq[i])]}')
                if((Dd != start) and (path_positions[path_positions.index(seq[i]) + 1] in seq)): # This is directly associated with 'In'/'Out', respectively.
                    try:
                        if(ROTATIONS[(start, Dd)] == 'Out'):


                            for r in R:
                                mountain_grid[r] = -90

                            print(f'{[pos_N for pos_N in get_neighbours(mountain_grid, r, diag = True) if mountain_grid[pos_N] > 0]}')
                            print(F'LOCATE: {path_positions[path_positions.index(seq[i]) + 1]}.')
                            print(f'ROTATIONS: {ROTATIONS[(start, Dd)]} {start} {Dd}')
                            print(f'POS: {start} {Dd}'); print(f'CYCLE = {cycle}.')
                            print(f'{R} is a counter-clockwise loop.'); print() # IN
                            in_Vec.append(R);
                            S += len(R)
                            break
                        else:

                            for r in R:
                                mountain_grid[r] = -20

                            print(f'{[pos_N for pos_N in get_neighbours(mountain_grid, r, diag = True) if mountain_grid[pos_N] > 0]}')
                            print(F'LOCATE: {path_positions[path_positions.index(seq[i]) + 1]}.')
                            # print(F'LOCATE: {locate_element(mountain_grid, sorted(num_seq)[i + 1])}.')
                            print(f'ROTATIONS: {ROTATIONS[(start, Dd)]} {start} {Dd}')
                            print(f'POS: {start} {Dd}'); print(f'CYCLE = {cycle}.')
                            print(f'{R} is a clockwise loop.'); print() # OUT
                            break
                    except:
                        start = Dd; print(F'START = {start}.')
                else:
                    print(f'rejected: LOCATE: {path_positions[path_positions.index(sorted(seq)[i])]} \
                          {path_positions[path_positions.index(seq[i]) + 1] in seq}. {Dd} {start} {Dd != start}.')

print(f'Number of tiles enclosed by the loop = {S}.')
# 20

SEQ = [(14, 26), (14, 25), (13, 25), (12, 25), (12, 26), (12, 27), (13, 27), (14, 27)].
SEQ = [(14, 26), (14, 25), (13, 25), (12, 25), (12, 26), (12, 27), (13, 27), (14, 27)].
position = (14, 26)
rejected: LOCATE: (12, 25)                           True. Left Left False.
position = (14, 25)
rejected: LOCATE: (12, 26)                           False. Left Left False.
position = (13, 25)
[(12, 26), (13, 27), (14, 26), (13, 25), (14, 27), (14, 25), (12, 27), (12, 25)]
LOCATE: (12, 25).
ROTATIONS: In Left Up
POS: Left Up
CYCLE = ['Left', 'Left', 'Up', 'Right', 'Up', 'Down', 'Down', 'Right'].
[(13, 26)] is a clockwise loop.

SEQ = [(13, 30), (12, 30), (12, 31), (12, 32), (13, 32), (14, 32), (14, 31), (14, 30)].
SEQ = [(13, 30), (12, 30), (12, 31), (12, 32), (13, 32), (14, 32), (14, 31), (14, 30)].
position = (13, 30)
rejected: LOCATE: (12, 30)                           True. Up Up False.
position = (12, 30)
rejected: LOCATE: (12, 31)                           False. Up Up False.
position = 

In [578]:
sorted(seq)

[(134, 49),
 (134, 50),
 (134, 51),
 (135, 49),
 (135, 51),
 (136, 49),
 (136, 50),
 (136, 51)]

Obtained final result, for part 2 = 455. Should be 371!

In [579]:
for row in mountain_grid:
    for value in row:
        print(f'{value:5.0f}', end=' ')
    print('\n')


   -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1 13573 13572 13571    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1 

   -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1    -1   

In [580]:
for row in direction_grid:
    for value in row:
        try:
            print(f'{value:5.0f}', end=' ')
        except:
            print(f'{value:5}', end=' ')
    print('\n')

    0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0 Down  Left  Left      0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0 

    0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   

In [581]:
import itertools

# Join all sublists into a single list
joined_list = list(itertools.chain.from_iterable(in_Vec))

# Print the joined list
print(joined_list)


[(22, 77), (23, 78), (36, 38), (36, 39), (36, 57), (36, 87), (36, 90), (37, 104), (38, 62), (38, 69), (39, 87), (39, 103), (41, 48), (41, 88), (42, 54), (42, 63), (43, 49), (43, 59), (44, 53), (44, 74), (44, 93), (44, 101), (44, 104), (45, 46), (46, 52), (46, 55), (46, 68), (47, 69), (46, 94), (48, 48), (48, 77), (49, 79), (50, 83), (50, 91), (51, 44), (51, 53), (51, 94), (51, 97), (52, 38), (53, 64), (53, 81), (54, 31), (55, 30), (54, 43), (54, 51), (55, 53), (55, 65), (55, 68), (56, 69), (57, 69), (58, 69), (58, 68), (59, 70), (59, 71), (60, 70), (60, 71), (59, 72), (60, 72), (58, 72), (61, 70), (61, 71), (61, 69), (61, 72), (61, 73), (57, 72), (62, 70), (62, 71), (62, 69), (62, 72), (61, 68), (62, 68), (62, 73), (61, 74), (62, 74), (63, 70), (63, 71), (63, 69), (63, 72), (63, 68), (63, 73), (63, 74), (61, 75), (62, 75), (63, 75), (64, 70), (64, 71), (64, 69), (64, 72), (64, 68), (64, 73), (64, 67), (64, 74), (64, 75), (61, 76), (62, 76), (63, 76), (64, 76), (65, 70), (65, 71), (65, 

In [582]:
seq

[(136, 49),
 (135, 49),
 (134, 49),
 (134, 50),
 (134, 51),
 (135, 51),
 (136, 51),
 (136, 50)]

In [583]:
sorted(seq, key=lambda x: path_positions.index(x))

[(136, 49),
 (135, 49),
 (134, 49),
 (134, 50),
 (134, 51),
 (135, 51),
 (136, 51),
 (136, 50)]

In [584]:
# Repopulating to '-1', from each corner. Aka, getting rid of the 'obvious' "Outsides"
L_m, L_n = len(mountain_grid) - 1, len(mountain_grid[0]) - 1
repopulate(0, 0, -1, False); repopulate(L_m, L_n, -1, False); repopulate(0, L_n, -1, False); repopulate(L_m, 0, -1, False)

S, pos = 0, None
for nx_pos in path_positions:
    neighbours = [pos_N for pos_N in get_neighbours(mountain_grid, nx_pos, diag = True) if mountain_grid[pos_N] == 0]
    if(len(neighbours) == 0): continue
    else:
        for n in neighbours:
            R = repopulate(n, -10, True); seq = []
            for r in R:
                seq.append([pos_N for pos_N in get_neighbours(mountain_grid, r, diag = True) if mountain_grid[pos_N] > 0])
            seq = set([item for sublist in seq for item in sublist]); cycle = [], []
            for p in path_positions:
                if(p in seq): cycle.append(direction_grid[p[0]][p[1]])
            #     for p in path_positions:
            # if(mountain_grid[n] == 1): pos = n; break
        break



    
    pos = nx_pos




for m in range(len(mountain_grid)):
    for n in range(len(mountain_grid[0])):
        if(mountain_grid[m, n] == 0):
            R = repopulate(m, n, -10, True); seq = []
            for r in R:
                print(f'{[pos_N for pos_N in get_neighbours(mountain_grid, r, diag = True)]}')
                seq.append([pos_N for pos_N in get_neighbours(mountain_grid, r, diag = True) if mountain_grid[pos_N] > 0])
            seq = set([item for sublist in seq for item in sublist]); num_seq, cycle = [], []
            for p in seq: num_seq.append(mountain_grid[p])    
            for value in sorted(num_seq):
                Pp = locate_element(mountain_grid, value)
                cycle.append(direction_grid[Pp[0]][Pp[1]])
            start = cycle[0]
            for Dd in cycle:     # Finds if the "circle path" is circulated in the 'clockwise' or 'counter-clockwise' direction.
                if(Dd != start): # This is directly associated with 'In'/'Out', respectively.
                    if(ROTATIONS[(start, Dd)] == 'In'):
                        S += len(R)
                        break
print(f'Number of tiles enclosed by the loop = {S}.')
# 20

Number of tiles enclosed by the loop = 0.


In [585]:
b = 5
for row in mountain_grid[47 - b:47 + b, 89 - b: 89 + b]:
    for value in row:
        print(f'{value:5.0f}', end=' ')
    print('\n')




11727 11698 11685 11686 11687 11234 11235 11236 11201 11190 

11728 11697 11692 11691 11688 11233 11226 11225 11202 11189 

11731 11696 11693 11690 11689 11232 11227 11224 11203   -90 

11732 11695 11694 11737 11738 11231 11228 11223 11204 11205 

11733 11734 11735 11736 11739 11230 11229 11222 11221 11206 

11772 11763 11762 11741 11740   -20 11218 11219 11220 11207 

11771 11764 11761 11742   -20 11216 11217 11212 11211 11208 

11770 11765 11760 11743 11744 11215 11214 11213 11210 11209 

11769 11766 11759 11758 11745 11746 11747   -90  9817  9818 

11768 11767 11756 11757 11750 11749 11748  9815  9816  9819 



In [586]:
mountain_grid[47, 89:92]


array([  -20., 11218., 11219.])